<a href="https://colab.research.google.com/github/dummydeveloper13/old_newspaper/blob/main/Spark_Old_Newspapers_Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING AND LIBRARIES

## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Old Newspapers Project')

## Downloading dataset with API

In [ ]:
# Install Kaggle API
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd //content/drive/MyDrive/Old Newspapers Project/

/content/drive/MyDrive/Old Newspapers Project


In [ ]:
# Configure directory to download dataset
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Old Newspapers Project"

In [ ]:
# Download dataset
!kaggle datasets download alvations/old-newspapers

100% 2.04G/2.05G [00:23<00:00, 88.6MB/s]
100% 2.05G/2.05G [00:23<00:00, 93.4MB/s]


In [ ]:
# Unzip dataset
!unzip '/content/drive/MyDrive/Old Newspapers Project/old-newspapers.zip' -d ''

Archive:  /content/drive/MyDrive/Old Newspapers Project/old-newspapers.zip
  inflating: old-newspaper.tsv       


## Packages and Libraries

In [ ]:
# Install package contractions
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Install package NLTK
!pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import string
import csv

In [ ]:
import contractions

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
# Download some of NLTK required packages
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Read and filter dataset

In [3]:
# Read dataset
df = pd.read_csv('/content/drive/MyDrive/Old Newspapers Project/old-newspaper.tsv', sep='\t', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: ignored

In [ ]:
df.head(10)

,Language,Source,Date,Text
0,Afrikaans,republikein.com.na,2011/09/14,Die veranderinge aan die Britsgeboude Avensis ...
1,Afrikaans,republikein.com.na,2011/01/20,Duitsland se mans- en vrouespanne is die afgel...
2,Afrikaans,sake24.com,2009/11/28,"Mnr. Estienne de Klerk, uitvoerende direkteur ..."
3,Afrikaans,sake24.com,2009/11/12,Mustek is se finansiële-resultate-advertensie ...
4,Afrikaans,sake24.com,2011/02/04,nadat LMS se raad van trustees in Junie verled...
5,Afrikaans,praag.co.za,2011/06/09,Hierdie hersirkulering van kaders werk net so ...
6,Afrikaans,rapport.co.za,2011/07/15,"Volgens ao. November Filander, polisiewoordvoe..."
7,Afrikaans,republikein.com.na,2011/05/20,Die plant is besonder gehard en kan selfs uite...
8,Afrikaans,republikein.com.na,2011/10/19,Dit volg op twee vorige ekspos deur SMEs Compe...
9,Afrikaans,praag.co.za,2009/06/01,Daarom moet 'n Afrikanerafvaardiging so gou mo...


In [ ]:
# Count number of languages
languages_count = df['Language'].nunique()
languages_count

66

In [ ]:
# Visualize the languages
languages = df['Language'].unique()
languages

array(['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian',
       'Azerbaijan', 'Bengali', 'Bosnian', 'Catalan',
       'Chinese (Simplified)', 'Chinese (Traditional)', 'Croatian',
       'Welsh', 'Czech', 'German', 'Danish', 'English', 'Spanish',
       'Spanish (South America)', 'Finnish', 'French', 'Georgian',
       'Galician', 'Greek', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic',
       'Indonesian', 'Italian', 'Japanese', 'Khmer', 'Kannada', 'Korean',
       'Kazakh', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam',
       'Mongolian', 'Malay', 'Nepali', 'Dutch', 'Norwegian (Bokmal)',
       'Punjabi', 'Farsi', 'Polish', 'Portuguese (Brazil)',
       'Portuguese (EU)', 'Romanian', 'Russian', 'Serbian', 'Sinhalese',
       'Slovak', 'Slovenian', 'Swahili', 'Swedish', 'Tamil', 'Telugu',
       'Tagalog', 'Thai', 'Turkish', 'Ukranian', 'Urdu', 'Uzbek',
       'Vietnamese'], dtype=object)

In [ ]:
# Filter dataset for English and remove unnecessary columns
dataset = df[(df.Language == 'English')]
dataset = dataset.drop(columns = ["Language", "Source", "Date"])
dataset.reset_index(drop=True, inplace=True)
dataset

In [ ]:
# Save dataset to csv
dataset.to_csv("/content/drive/MyDrive/Old Newspapers Project/dataset.csv")

# PRE-PROCESSING

## Replace special characters and expand contractions

In [ ]:
# Read dataset
dataset = pd.read_csv("/content/drive/MyDrive/Old Newspapers Project/dataset.csv")

In [ ]:
# Write the dataset as a list
baskets = []

for i in range(dataset.shape[0]):
  baskets.append(str(dataset.Text[i]))

In [ ]:
# Replace special characters and expand contractions
baskets_ext = []

for sentence in baskets:
  sentence = sentence.replace("\'s", "")
  sentence = sentence.replace("\'", "")
  sentence = sentence.replace("/'", "")
  sentence = sentence.replace("/", " ")
  sentence = sentence.replace("-", " ")
  sentence = sentence.replace("ø", "")
  baskets_ext.append(contractions.fix(sentence))

In [ ]:
baskets_ext

['He was not home alone, apparently.',
 'The St. Louis plant had to close. It would die of old age. Workers had been making cars there since the onset of mass automotive production in the 1920s.',
 'WSU plans quickly became a hot topic on local online sites. Though most people applauded plans for the new biomedical center, many deplored the potential loss of the building.',
 'The Alaimo Group of Mount Holly was up for a contract last fall to evaluate and suggest improvements to Trenton Water Works. But campaign finance records released this week show the two employees donated a total of $4,500 to the political action committee (PAC) Partners for Progress in early June. Partners for Progress reported it gave more than $10,000 in both direct and in kind contributions to Mayor Tony Mack in the two weeks leading up to his victory in the mayoral runoff election June 15.',
 'And when it often difficult to predict a law impact, legislators should think twice before carrying any bill. Is it ab

## Split sentences

In [ ]:
# Split sentences and create a list of lists
itemset_1 = []

for sentence in baskets_ext:
  itemset_1.append(sentence.split(' '))
itemset_1[:10]

[['He', 'was', 'not', 'home', 'alone,', 'apparently.'],
 ['The',
  'St.',
  'Louis',
  'plant',
  'had',
  'to',
  'close.',
  'It',
  'would',
  'die',
  'of',
  'old',
  'age.',
  'Workers',
  'had',
  'been',
  'making',
  'cars',
  'there',
  'since',
  'the',
  'onset',
  'of',
  'mass',
  'automotive',
  'production',
  'in',
  'the',
  '1920s.'],
 ['WSU',
  'plans',
  'quickly',
  'became',
  'a',
  'hot',
  'topic',
  'on',
  'local',
  'online',
  'sites.',
  'Though',
  'most',
  'people',
  'applauded',
  'plans',
  'for',
  'the',
  'new',
  'biomedical',
  'center,',
  'many',
  'deplored',
  'the',
  'potential',
  'loss',
  'of',
  'the',
  'building.'],
 ['The',
  'Alaimo',
  'Group',
  'of',
  'Mount',
  'Holly',
  'was',
  'up',
  'for',
  'a',
  'contract',
  'last',
  'fall',
  'to',
  'evaluate',
  'and',
  'suggest',
  'improvements',
  'to',
  'Trenton',
  'Water',
  'Works.',
  'But',
  'campaign',
  'finance',
  'records',
  'released',
  'this',
  'week',
  's

## Remove punctuation, blank spaces and capitalization

In [ ]:
punctuation = string.punctuation
punctuation = punctuation.replace("'", "")
punctuation

'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# Remove punctuation, English possessive, spaces and capitalization
itemset_2 = []
for idx, sentence in enumerate(itemset_1):
  itemset_2.append([])
  for word in sentence:
    new_word = word.translate(str.maketrans('', '', punctuation))
    new_word = new_word.replace("’s", "")
    new_word = new_word.replace("–","")
    new_word = new_word.strip()
    new_word = new_word.lower()
    if new_word != "":
          itemset_2[idx].append(new_word)

itemset_2[:10]

[['he', 'was', 'not', 'home', 'alone', 'apparently'],
 ['the',
  'st',
  'louis',
  'plant',
  'had',
  'to',
  'close',
  'it',
  'would',
  'die',
  'of',
  'old',
  'age',
  'workers',
  'had',
  'been',
  'making',
  'cars',
  'there',
  'since',
  'the',
  'onset',
  'of',
  'mass',
  'automotive',
  'production',
  'in',
  'the',
  '1920s'],
 ['wsu',
  'plans',
  'quickly',
  'became',
  'a',
  'hot',
  'topic',
  'on',
  'local',
  'online',
  'sites',
  'though',
  'most',
  'people',
  'applauded',
  'plans',
  'for',
  'the',
  'new',
  'biomedical',
  'center',
  'many',
  'deplored',
  'the',
  'potential',
  'loss',
  'of',
  'the',
  'building'],
 ['the',
  'alaimo',
  'group',
  'of',
  'mount',
  'holly',
  'was',
  'up',
  'for',
  'a',
  'contract',
  'last',
  'fall',
  'to',
  'evaluate',
  'and',
  'suggest',
  'improvements',
  'to',
  'trenton',
  'water',
  'works',
  'but',
  'campaign',
  'finance',
  'records',
  'released',
  'this',
  'week',
  'show',
  't

## Remove digits

In [ ]:
# Remove digits and words containing digits
itemset_3 = []
for idx, sentence in enumerate(itemset_2):
  itemset_3.append([])
  for word in sentence:
    if any(c.isdigit() for c in word) == False: 
      itemset_3[idx].append(word)

itemset_3[:10]

[['he', 'was', 'not', 'home', 'alone', 'apparently'],
 ['the',
  'st',
  'louis',
  'plant',
  'had',
  'to',
  'close',
  'it',
  'would',
  'die',
  'of',
  'old',
  'age',
  'workers',
  'had',
  'been',
  'making',
  'cars',
  'there',
  'since',
  'the',
  'onset',
  'of',
  'mass',
  'automotive',
  'production',
  'in',
  'the'],
 ['wsu',
  'plans',
  'quickly',
  'became',
  'a',
  'hot',
  'topic',
  'on',
  'local',
  'online',
  'sites',
  'though',
  'most',
  'people',
  'applauded',
  'plans',
  'for',
  'the',
  'new',
  'biomedical',
  'center',
  'many',
  'deplored',
  'the',
  'potential',
  'loss',
  'of',
  'the',
  'building'],
 ['the',
  'alaimo',
  'group',
  'of',
  'mount',
  'holly',
  'was',
  'up',
  'for',
  'a',
  'contract',
  'last',
  'fall',
  'to',
  'evaluate',
  'and',
  'suggest',
  'improvements',
  'to',
  'trenton',
  'water',
  'works',
  'but',
  'campaign',
  'finance',
  'records',
  'released',
  'this',
  'week',
  'show',
  'the',
  'two

## Remove stopwords

In [ ]:
# Remove the majority of stopwords
stop_words = set(stopwords.words('english'))
new_stopwords = ['i', 'us']
new_stopwords_list = stop_words.union(new_stopwords)

itemset_4 = []
for idx, sentence in enumerate(itemset_3):
  itemset_4.append([])
  for word in sentence:
    if not word in stop_words: 
      itemset_4[idx].append(word)

itemset_4[:10]

[['home', 'alone', 'apparently'],
 ['st',
  'louis',
  'plant',
  'close',
  'would',
  'die',
  'old',
  'age',
  'workers',
  'making',
  'cars',
  'since',
  'onset',
  'mass',
  'automotive',
  'production'],
 ['wsu',
  'plans',
  'quickly',
  'became',
  'hot',
  'topic',
  'local',
  'online',
  'sites',
  'though',
  'people',
  'applauded',
  'plans',
  'new',
  'biomedical',
  'center',
  'many',
  'deplored',
  'potential',
  'loss',
  'building'],
 ['alaimo',
  'group',
  'mount',
  'holly',
  'contract',
  'last',
  'fall',
  'evaluate',
  'suggest',
  'improvements',
  'trenton',
  'water',
  'works',
  'campaign',
  'finance',
  'records',
  'released',
  'week',
  'show',
  'two',
  'employees',
  'donated',
  'total',
  'political',
  'action',
  'committee',
  'pac',
  'partners',
  'progress',
  'early',
  'june',
  'partners',
  'progress',
  'reported',
  'gave',
  'direct',
  'kind',
  'contributions',
  'mayor',
  'tony',
  'mack',
  'two',
  'weeks',
  'leading',

## Lemmatize words

In [ ]:
# Lemmatize words 
# Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item.
# Lemmatization is preferred over Stemming because lemmatization does morphological analysis of the words
lemmatizer = WordNetLemmatizer()
itemset_5 = []
for idx, sentence in enumerate(itemset_4):
  itemset_5.append([])
  for word in sentence:
    rootWord = lemmatizer.lemmatize(word)
    itemset_5[idx].append(rootWord)

itemset_5[:10]

[['home', 'alone', 'apparently'],
 ['st',
  'louis',
  'plant',
  'close',
  'would',
  'die',
  'old',
  'age',
  'worker',
  'making',
  'car',
  'since',
  'onset',
  'mass',
  'automotive',
  'production'],
 ['wsu',
  'plan',
  'quickly',
  'became',
  'hot',
  'topic',
  'local',
  'online',
  'site',
  'though',
  'people',
  'applauded',
  'plan',
  'new',
  'biomedical',
  'center',
  'many',
  'deplored',
  'potential',
  'loss',
  'building'],
 ['alaimo',
  'group',
  'mount',
  'holly',
  'contract',
  'last',
  'fall',
  'evaluate',
  'suggest',
  'improvement',
  'trenton',
  'water',
  'work',
  'campaign',
  'finance',
  'record',
  'released',
  'week',
  'show',
  'two',
  'employee',
  'donated',
  'total',
  'political',
  'action',
  'committee',
  'pac',
  'partner',
  'progress',
  'early',
  'june',
  'partner',
  'progress',
  'reported',
  'gave',
  'direct',
  'kind',
  'contribution',
  'mayor',
  'tony',
  'mack',
  'two',
  'week',
  'leading',
  'victory',

## Remove duplicates

In [ ]:
from collections import OrderedDict
itemset_6 = []
for sentence in enumerate(itemset_5):
  new_sentence = list(OrderedDict.fromkeys(sentence[1]))
  itemset_6.append(new_sentence)
itemset_6[:10]

[['home', 'alone', 'apparently'],
 ['st',
  'louis',
  'plant',
  'close',
  'would',
  'die',
  'old',
  'age',
  'worker',
  'making',
  'car',
  'since',
  'onset',
  'mass',
  'automotive',
  'production'],
 ['wsu',
  'plan',
  'quickly',
  'became',
  'hot',
  'topic',
  'local',
  'online',
  'site',
  'though',
  'people',
  'applauded',
  'new',
  'biomedical',
  'center',
  'many',
  'deplored',
  'potential',
  'loss',
  'building'],
 ['alaimo',
  'group',
  'mount',
  'holly',
  'contract',
  'last',
  'fall',
  'evaluate',
  'suggest',
  'improvement',
  'trenton',
  'water',
  'work',
  'campaign',
  'finance',
  'record',
  'released',
  'week',
  'show',
  'two',
  'employee',
  'donated',
  'total',
  'political',
  'action',
  'committee',
  'pac',
  'partner',
  'progress',
  'early',
  'june',
  'reported',
  'gave',
  'direct',
  'kind',
  'contribution',
  'mayor',
  'tony',
  'mack',
  'leading',
  'victory',
  'mayoral',
  'runoff',
  'election'],
 ['often',
  'd

## Save

In [ ]:
# Save pre-processed words to csv with index
with open("/content/drive/MyDrive/Old Newspapers Project/itemsets.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(itemset_6)

In [ ]:
with open("/content/drive/MyDrive/Old Newspapers Project/itemsets.csv", 'r') as read_obj:
    # Return a reader object which will
    # iterate over lines in the given csvfile
    csv_reader = csv.reader(read_obj)
  
    # convert string to list
    list_of_csv = list(csv_reader)

In [ ]:
dataset = pd.DataFrame(list_of_csv)


In [ ]:
dataset = dataset.T

In [ ]:
dataset.head()

,0
0,"['home', 'alone', 'apparently']"
1,"['st', 'louis', 'plant', 'close', 'would', 'di..."
2,"['wsu', 'plan', 'quickly', 'became', 'hot', 't..."
3,"['alaimo', 'group', 'mount', 'holly', 'contrac..."
4,"['often', 'difficult', 'predict', 'law', 'impa..."


In [ ]:
dataset.to_csv("/content/drive/MyDrive/Old Newspapers Project/dataframe.csv",index=True, header=False)

# FREQUENT ITEMSETS WITH SPARK

## Setting up pyspark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 43.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=6219ac04dd3d13ffc435974335f74bdd7332dfbd1c20824d90c1098df022e186
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
!pip install findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import SparkSession 
from pyspark.sql import SparkSession
# Create a Spark Session 
spark = SparkSession.builder.appName("Newspapers").master("local").getOrCreate()
# Check Spark Session Information 
spark


In [ ]:
sc = spark.sparkContext

In [ ]:
from pyspark.rdd import RDD

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Apriori Algorithm

### Load dataset in spark

In [ ]:
# Reading Data Set with PySpark
lines = sc.textFile("/content/drive/MyDrive/Old Newspapers Project/itemsets.csv")

In [ ]:
print(lines.take(10))

['home,alone,apparently', 'st,louis,plant,close,would,die,old,age,worker,making,car,since,onset,mass,automotive,production', 'wsu,plan,quickly,became,hot,topic,local,online,site,though,people,applauded,plan,new,biomedical,center,many,deplored,potential,loss,building', 'alaimo,group,mount,holly,contract,last,fall,evaluate,suggest,improvement,trenton,water,work,campaign,finance,record,released,week,show,two,employee,donated,total,political,action,committee,pac,partner,progress,early,june,partner,progress,reported,gave,direct,kind,contribution,mayor,tony,mack,two,week,leading,victory,mayoral,runoff,election,june', 'often,difficult,predict,law,impact,legislator,think,twice,carrying,bill,absolutely,necessary,issue,serious,enough,merit,attention,definitely,make,situation,worse', 'certain,amount,scoffing,going,around,year,ago,nfl,decided,move,draft,weekend,prime,time,eventually,splitting,first,round,separate,day', 'charlevoix,detroit', 'another,long,line,failed,attempt,subsidize,atlantic,city

In [ ]:
lines.getNumPartitions()

6

### Phase-1


In [ ]:
def sumOperator(x,y):
    return x+y

In [ ]:
# Parsing items into RDD
lblitems = lines.mapPartitions(lambda line: line.split(','))

In [ ]:
#Randomly sample 10% of the data without replacement
lblitems = lblitems.sample(False, 0.1, seed=0)

In [ ]:
# Split into RDD of single words
wlitems = lines.flatMap(lambda line:line.split(','))

In [ ]:
print(wlitems.take(10))

['home', 'alone', 'apparently', 'st', 'louis', 'plant', 'close', 'would', 'die', 'old']


In [ ]:
## Calculate support for each item
# Unique frequent items in dataset
uniqueItems = wlitems.distinct()
# Add 1 as Tuple
supportRdd = wlitems.mapPartitions(lambda item: (item , 1))

# Sum of values by key
supportRdd = supportRdd.reduceByKey(sumOperator, 10)

In [ ]:
# First support values
supports = supportRdd.mapPartitions(lambda item: item[1]) # Return only support values

In [ ]:
# Define minimum support value 
minSupport = 0.01*100000

# Filter first supportRdd with minimum support 
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

# Create base RDD with will be updated every iteration
baseRdd = supportRdd.mapPartitions(lambda item: ([item[0]] , item[1])) 
print('1 . Table has been created...') 

supportRdd = supportRdd.mapPartitions(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has been created...


### Phase-2

In [ ]:
# Define function to remove replicas
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [ ]:
# the first support table will have item-sets which has 2 items 

combined = supportRdd.cartesian(uniqueItems)
combined = combined.mapPartitions(lambda item: removeReplica(item))
combined = combined.filter(lambda item: len(item) == 2)

In [ ]:
combined_2 = combined.mapPartitions(lambda item: item[0])
combined_2 = combined_2.mapPartitions(lambda item: (item , 1))
combined_2 = combined_2.reduceByKey(sumOperator, 10)
combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

In [ ]:
baseRdd = baseRdd.union(combined_2)

In [ ]:
combined_2 = combined_2.mapPartitions(lambda item: item[0])
supportRdd = combined_2
print('2. Table has crated... ')

2. Table has crated... 


In [ ]:
# Filter catersian RDD according to conditions 
# Condition 1- If both tuple elements lenght are same 
# Condition 2- Check it is a tuple or single str object (which first table that show single items supports)

class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

In [ ]:
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

In [ ]:
# Create Filter Object
ff = Filter()

In [ ]:
total = calcuItems.count()

In [ ]:
print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item, total))

In [ ]:
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.mapPartitions(lambda item: ff.calculateConfidence(item))

In [ ]:
## Import pandas modules
import pandas as pd

## Create an array with collected baseRddConfidence results
result = baseRddConfidence.collect()

## Create Data Frame
confidenceTable = pd.DataFrame(data = result , columns=["Before", "After" , "Confidence"])

## Show data frame
print(confidenceTable.sort_values(by=['Confidence'], ascending = False))

## FP-Growth Algorithm

### Load dataset in spark

In [ ]:
# Reading Data Set with PySpark
path = "/content/drive/MyDrive/Old Newspapers Project/dataframe.csv"
schema = StructType([\
    StructField("id", IntegerType(), True),\
    StructField("items", StringType(), True)])
dataset = spark.read.schema(schema).csv(path)

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- items: string (nullable = true)



In [ ]:
df = dataset.withColumn("items", split(regexp_replace(col("items"), '[\\[\\]]', ""), ","))

In [ ]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
df.show()

+---+--------------------+
| id|               items|
+---+--------------------+
|  0|['home',  'alone'...|
|  1|['st',  'louis', ...|
|  2|['wsu',  'plan', ...|
|  3|['alaimo',  'grou...|
|  4|['often',  'diffi...|
|  5|['certain',  'amo...|
|  6|['charlevoix',  '...|
|  7|['another',  'lon...|
|  8|['time',  'report...|
|  9|['trying',  'hit'...|
| 10|['mhta',  'presid...|
| 11|['absurdity',  'a...|
| 12|['gm',  'labor', ...|
| 13|['wandry',  'matt...|
| 14|['cheap',  'said'...|
| 15|['andrade',  'chi...|
| 16|['let',  'hair', ...|
| 17|['born',  'april'...|
| 18|['house',  'minor...|
| 19|['first',  'love'...|
+---+--------------------+
only showing top 20 rows



### Model

In [ ]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.02, minConfidence=0.02)
model = fpGrowth.fit(df)

### Results

In [ ]:
# Display frequent itemsets.
items = model.freqItemsets

In [ ]:
items.show()

+--------------+-----+
|         items| freq|
+--------------+-----+
|     [ 'give']|13403|
|    [ 'among']|10668|
|    [ 'night']|17597|
|    [ 'event']|11121|
|     [ 'york']|11943|
|   [ 'health']|12779|
|   [ 'office']|15790|
|   [ 'better']|14466|
|    [ 'world']|16258|
|    [ 'never']|14725|
|     [ 'room']|11430|
|    [ 'early']|12198|
|  [ 'service']|18020|
|   [ 'change']|12885|
|    [ 'point']|22520|
|[ 'including']|19745|
|   [ 'season']|27323|
| [ 'national']|13620|
|      [ 'run']|20653|
|[ 'community']|13948|
+--------------+-----+
only showing top 20 rows



In [ ]:
# Display generated association rules.
rules = model.associationRules

In [ ]:
rules.show()

+----------+----------+-------------------+------------------+--------------------+
|antecedent|consequent|         confidence|              lift|             support|
+----------+----------+-------------------+------------------+--------------------+
|[ 'would']| [ 'said']| 0.3343840137976152|1.5113134165485431|0.022261992671062974|
| [ 'said']| [ 'year']|0.10348962061560486|0.9916960996912486| 0.02289748396918758|
| [ 'said']|[ 'would']|0.10061739441660701| 1.511313416548543|0.022261992671062974|
| [ 'year']| [ 'said']|0.21941664690538298|0.9916960996912487| 0.02289748396918758|
+----------+----------+-------------------+------------------+--------------------+

